# Revenue Data

Source=DNP a partir de información del FUT y DANE

In [8]:
import pandas as pd
import numpy as np

In [3]:
rev=pd.read_excel('TerriData_Dim7_Sub1.xlsx')

In [9]:
rev.drop(index=0,inplace=True)

In [17]:
# filter to get only revenues
revenues=rev[rev['Indicador']=='Regalías per cápita (Valor efectivamente girado al municipio)']

# drop at level of colombia
revenues=revenues[revenues.Departamento != 'Colombia']

revenues.head(2)

,Código Departamento,Departamento,Código Entidad,Entidad,Dimensión,Subcategoría,Indicador,Dato Numérico,Dato Cualitativo,Año,Mes,Fuente,Unidad de Medida
1221,5.0,Antioquia,5000.0,Antioquia,Finanzas públicas,Operaciones efectivas de caja,Regalías per cápita (Valor efectivamente girad...,"7.937,84",NaN,2000.0,12.0,DNP a partir de información del FUT y DANE,Pesos corrientes
1222,5.0,Antioquia,5000.0,Antioquia,Finanzas públicas,Operaciones efectivas de caja,Regalías per cápita (Valor efectivamente girad...,"10.988,11",NaN,2001.0,12.0,DNP a partir de información del FUT y DANE,Pesos corrientes


In [24]:
# keep relevant columns
revenues=revenues[['Entidad','Código Entidad','Dato Numérico','Año']]

# rename columns 
revenues.rename(columns={'Entidad':'munname','Código Entidad':'muncode','Dato Numérico':'CapitalRevenue','Año':'year'},inplace=True)

In [38]:
revenues.CapitalRevenue=revenues.CapitalRevenue.str.replace('.','')
revenues.CapitalRevenue=revenues.CapitalRevenue.str.replace(',','.')
revenues.CapitalRevenue=revenues.CapitalRevenue.astype(float)

C:\Users\anasa\AppData\Local\Temp\ipykernel_28684\1894758209.py:1: FutureWarning: The default value of regex will change from True to False in a future version. In addition, single character regular expressions will *not* be treated as literal strings when regex=True.
  revenues.CapitalRevenue=revenues.CapitalRevenue.str.replace('.','')


In [40]:
revenues.to_csv('revenues.csv')

# Turn to 2006 pesos

In [6]:
revenues=pd.read_csv('revenues.csv',index_col=0)

In [10]:
revenues.head(2)

,munname,muncode,CapitalRevenue,year
1221,Antioquia,5000.0,793784.0,2000.0
1222,Antioquia,5000.0,1098811.0,2001.0


In [58]:
# turn capital revenue in 2006 pesos 

# cpi index
cpi=pd.read_csv("CPI_colombia.csv")

# keep relevant columns 
cpi=cpi[['Unit of measure','TIME_PERIOD','OBS_VALUE']]

# only information on CPI index 
cpi=cpi[cpi['Unit of measure']=='Index'].reset_index().drop(['index','Unit of measure'],axis=1)

# rename columns 
cpi.rename(columns={'TIME_PERIOD':'year','OBS_VALUE':'CPI'}, inplace=True)

# extract 2006 CPI
cpi_2006=cpi[cpi['year']==2006]['CPI'].values

# create cpi tranformer column
cpi['frac']=cpi_2006/cpi['CPI']

# merge cpi to data
rev_2006=pd.merge(revenues,cpi,how='left',on='year',validate='m:1')

# turn revenues to 2006 pesos
rev_2006['caprev_2006_percapita']=rev_2006['CapitalRevenue']*rev_2006['frac']

# drop unnecessary columns 
rev_2006.drop(columns=['CPI','frac'],inplace=True)

In [62]:
rev_2006.drop(columns='munname',inplace=True)

In [63]:
rev_2006.to_csv('revenue_2006.csv')